# This notebook is to create a Markov State Model for N-terminal Domain of HIV-1 Integrase (IN) using the PyEMMA software package

To learn how to install and use PyEMMA, please go to http://emma-project.org/latest/. This package was introduced by the Noé group:

*Scherer, M. K., Trendelkamp-Schroer, B., Paul, F., Pérez-Hernández, G., Hoffmann, M., Plattner, N., ... & Noé, F. (2015). PyEMMA 2: A software package for estimation, validation, and analysis of Markov models. Journal of chemical theory and computation, 11(11), 5525-5542.*

All-atom simulation trajectory for IN was published in the following article:

*Piana, S., Donchev, A. G., Robustelli, P., & Shaw, D. E. (2015). Water dispersion interactions strongly influence simulated structural properties of disordered protein states. The journal of physical chemistry B, 119(16), 5113-5123.*

and kindly provided by DE Shaw Research Group

In [ ]:
#Load Libraries and Modules

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import mdshare
import pyemma
from pyemma.util.contexts import settings

In [ ]:
pdb = 'Integrase.pdb' #Integrase structure imported from PDB (entry 1WJB)

files = ['Integrase-0.dcd','Integrase-1.dcd','Integrase-2.dcd','Integrase-3.dcd'] 
#DESRES trajectories of 40µs all-atom simulation 
#(translationally and rotationally aligned to reference structure in VMD)

In [ ]:
#PyEMMA allows use of several kinds of featurization capabilities

torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True, periodic=False)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)
labels = ['backbone\ntorsions']

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Ca())
positions_data = pyemma.coordinates.load(files, features=positions_feat)
labels += ['Ca atom\npositions']

distances_feat = pyemma.coordinates.featurizer(pdb)
distances_feat.add_distances(
    distances_feat.pairs(distances_feat.select_Ca(), excluded_neighbors=2), periodic=False)
distances_data = pyemma.coordinates.load(files, features=distances_feat)
labels += ['Ca atom\ndistances']

In [ ]:
#We chose C-alpha atom positions as our feature-set for TICA transformation

tica = pyemma.coordinates.tica(positions_data, lag=12)
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

In [ ]:
#The TICA-transformed coordinates are now clustered into discrete states using a k-means algorithm

cluster = pyemma.coordinates.cluster_kmeans(
    tica_output, k=350, max_iter=50, stride=1, fixed_seed=1)
dtrajs_concatenated = np.concatenate(cluster.dtrajs)

In [ ]:
#A reversible MSM is estimated for the clusters with the lag-time optimized such that the end-to-end distance
#autocorrelation function (our observable) predicted by it matches with that obtained from the data (see below)

msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, reversible=True, lag=21, dt_traj='1 ns')
print('fraction of states used = {:.2f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:.2f}'.format(msm.active_count_fraction))

In [ ]:
#End-to-end distance from original trajectory

import pyemma.coordinates as coor

feat = coor.featurizer(pdb)
feat.add_distances(np.array([[1,890]]), periodic=False)
D = coor.load(files, feat)

D=np.concatenate(tuple(D))
D=np.concatenate(tuple(D))

In [ ]:
#End-to-end distance reconstructed using MSM

dtrajs=cluster.dtrajs
dtrajs[0]

def average_by_state(dtraj, x, nstates):
    assert(len(dtraj) == len(x))
    N = len(dtraj)
    res = np.zeros((nstates))
    for i in range(nstates):
        I = np.argwhere(dtraj == i)[:,0]
        res[i] = np.mean(x[I])
    return res

dmean = average_by_state(dtrajs_concatenated, D, msm.nstates)

dMSM = np.array([dmean[state] for state in dtrajs_concatenated])

plt.plot(D, linewidth=2)
plt.plot(dMSM, linewidth=2)
plt.xlabel('time / ns')
plt.ylabel('D')

In [ ]:
#Compare end-to-end distance autocorrelation functions

print(msm.expectation(dmean)**2)
print(np.mean(D)**2)
print(msm.expectation(dmean**2))
print(np.mean(D**2))

corr_exact = np.correlate(D,D,"full")[:len(D/2)]
corr_exact=corr_exact[::-1]

corr_exact = [y/(len(corr_exact)-x) for (x,y) in enumerate(corr_exact)]

length=1500
frac=1

poscor=corr_exact[:length]
times_exact=np.array([x for x in range(length)])
plt.plot(times_exact, poscor, linewidth=2)
plt.xlabel('time / ns')
plt.ylabel('autocorrelation')

times, corr = msm.correlation(dmean, maxtime=1500)
times=times[:int(length*frac)]
corr=corr[:int(length*frac)]
plt.plot(times, corr, linewidth=2)
plt.xlabel('time / ns')
plt.ylabel('autocorrelation')

plt.plot(times, [msm.expectation(dmean)**2]*len(times), linewidth=2)

plt.plot(times, [np.mean(D)**2]*len(times), linewidth=2)

Thus, we have constructed an MSM that successfully predicts the end-to-end distance correlation function observed in the data

In [ ]:
# Calculate rate matrix for analysis

from msmtools.estimation import count_matrix, rate_matrix
c=count_matrix(dtrajs_concatenated, lag)

rmethCVE=rate_matrix(c.toarray(), lag, method='CVE')
rmethCVE[:3,:3]

In [ ]:
#Write to file

with open('rmethCVE-MSM_IN_TICAlag12_k350_lag21.dat','wb') as f:
    np.savetxt(f, rmethCVE, fmt='%.12f')

In [ ]:
#Demarcate the transition region (TR) states from states to the left (L) of the TR and to right (R)

trTable=np.array([i for (i,num) in enumerate(dmean) if num>2 and num<6])

with open('trTable-MSM_IN_TICAlag12_k350_lag21.dat','wb') as f:
    np.savetxt(f, trTable, fmt='%d')
    
rTable=np.array([i for (i,num) in enumerate(dmean) if num>6])

with open('rTable-MSM_IN_TICAlag12_k350_lag21.dat','wb') as f:
    np.savetxt(f, rTable, fmt='%d')
    
lTable=np.array([i for (i,num) in enumerate(dmean) if num<2])

with open('lTable-MSM_IN_TICAlag12_k350_lag21.dat','wb') as f:
    np.savetxt(f, lTable, fmt='%d')

Next, follow the Mathematica notebook to extract transition path times